In [ ]:
# TODO Tuto for using / implementing a variant in ACETONE

# Importing the package

In [ ]:
import acetone_nnet

# Generating code, example ACAS

There is two way to generate the code:
    -Using the function 'cli_acetone' to directly generate both the output python and the code C
    -Using the class 'CodeGenerator' to have more controle on the generation


This method is mainly used as a command-line, either by runing the python file, either by using the built in command: acetone_generate.
Confere to the ReadMe for example using a terminal.
This method is prefered when using the package. 
It allows more regarding the type of the arguments, give more controle over the generation.

### Defining the constants to use

In [ ]:
# Path to the model whose code you want to generate, or directly the model to parse (either Onnx or Keras model, created using their python package)
model_path = "../tests/models/acas/acas_COC/nn_acas_COC.nnet" 
test_dataset = None  # Replace by the path to the dataset on which the test must be done, or by a numpy array containing the dataset. If None, the generator while create a random dataset
function_name = "demo_acas"  # Replace by the name you want to give to the generated function (default is 'inference')
nb_tests = 1  # Replace by the number of tests you want to run
output_path = "demo_acas_nnet"  # Replace by the path to directory in which the code C while be generated (must be an already existing folder)

### Instantiating a 'CodeGenerator' element

In [ ]:
generator = acetone_nnet.CodeGenerator(file=model_path,
                                       test_dataset=test_dataset,
                                       function_name=function_name,
                                       nb_tests=nb_tests)

Finished model initialization.


### Generating the C code

In [ ]:
generator.generate_c_files(output_path)

Generated function source file.
Generated function header file.
Generated globalvars .c file.
Generated main file.
Generated Makefile.
Generated testdataset files.


### Generating the Python output

In [ ]:
generator.compute_inference(output_path)

File output_python.txt generated.


array([24715.65515468, 24798.92444016, 24650.51800126, 24824.74409792,
       24628.39230361])

### Importing the ONNX model

In [ ]:
model_path = "../tests/models/acas/acas_COC/nn_acas_COC.onnx"
output_path = "demo_acas_onnx"

onnx_generator = acetone_nnet.CodeGenerator(file=model_path,
                                       test_dataset=generator.test_dataset,
                                       function_name=function_name,
                                       nb_tests=nb_tests)

Finished model initialization.


In [ ]:
generator.generate_c_files(output_path)

Generated function source file.
Generated function header file.
Generated globalvars .c file.
Generated main file.
Generated Makefile.
Generated testdataset files.


# Generating code for LeNet with variants

In [ ]:
model_path = "../tests/models/lenet5/lenet5_trained/lenet5_trained.json"  # Replace by the path to the model whose code you want to generate, or directly by the model to parse (either Onnx or Keras model, created using their python package)
test_dataset = None  # Replace by the path to the dataset on which the test must be done, or by a numpy array containing the dataset. If None, the generator while create a random dataset
function_name = "demo_lenet5"  # Replace by the name you want to give to the generated function (default is 'inference')
nb_tests = 1  # Replace by the number of tests you want to run
conv_algorithm = "indirect_gemm_nn"  # Replace by the implementation of the Conv algorithm you desire
output_path = "demo_lenet5"  # Replace by the path to directory in which the code C while be generated

generator = acetone_nnet.CodeGenerator(file=model_path,
                                       test_dataset=test_dataset,
                                       function_name=function_name,
                                       nb_tests=nb_tests,
                                       versions={"Conv2D": conv_algorithm},)

generator.generate_c_files(output_path)

Finished model initialization.
Generated function source file.
Generated function header file.
Conv2D 1 patches size: (14400,)
Conv2D 3 patches size: (9600,)
Generated globalvars .c file.
Generated main file.
Generated Makefile.
Generated testdataset files.


In [ ]:
# TODO Run

In [ ]:
model_path = "../tests/models/lenet5/lenet5_trained/lenet5_trained.json"  # Replace by the path to the model whose code you want to generate, or directly by the model to parse (either Onnx or Keras model, created using their python package)
test_dataset = None  # Replace by the path to the dataset on which the test must be done, or by a numpy array containing the dataset. If None, the generator while create a random dataset
function_name = "demo_lenet5"  # Replace by the name you want to give to the generated function (default is 'inference')
nb_tests = 1  # Replace by the number of tests you want to run
conv_algorithm = "std_gemm_nn"  # Replace by the implementation of the Conv algorithm you desire
output_path = "demo_lenet5_gemm"  # Replace by the path to directory in which the code C while be generated

generator = acetone_nnet.CodeGenerator(file=model_path,
                                       test_dataset=generator.test_dataset,
                                       function_name=function_name,
                                       nb_tests=nb_tests,
                                       versions={"Conv2D": conv_algorithm})

generator.generate_c_files(output_path)

Finished model initialization.
Generated function source file.
Generated function header file.
Generated globalvars .c file.
Generated main file.
Generated Makefile.
Generated testdataset files.


# Compiling and running the generated code

Alongside the neural network's code, a main file is generated as a way to run and test the code. The provided Makefile gives the flags to use for the compilation.

In [ ]:
! make -C demo_acas_nnet all

make : on entre dans le répertoire « /tmp_user/ldtis506h/yaitaiss/acetone/examples/demo_acas_nnet »
gcc  -g -w -lm   -c -o main.o main.c
gcc   -o demo_acas inference.o global_vars.o main.o test_dataset.o  inference.h test_dataset.h   -g -w -lm
make : on quitte le répertoire « /tmp_user/ldtis506h/yaitaiss/acetone/examples/demo_acas_nnet »


To run the executable file, add as parameter the path to the text file in which the ouptut will be written. 

In [ ]:
! ./demo_acas_nnet/demo_acas ./demo_acas_nnet/output_c.txt

   Average time over 1 tests: 8.600000e-07 s 
   ACETONE framework's inference output: 
24715.6562 24798.9219 24650.5176 24824.7441 24628.3906 


# Comparing two ouptuts

In [ ]:
! acetone_compare ./demo_acas_nnet/output_python.txt ./demo_acas_nnet/output_c.txt 1

    Max absolute error for 1 test(s): 0.001953125
    Max relative error for 1 test(s): 7.930380144358296e-08



In [ ]:
! acetone_compare ./demo_acas_onnx/output_acas.txt ./demo_acas_nnet/output_c.txt 1

    Max absolute error for 1 test(s): 0.0
    Max relative error for 1 test(s): 0.0



# TODO
- Delete generated code before generation
- Rename the nnet to nnet
- Add calls to
  - compiler
  - run
  - compare nnet and onnx outputs in C
  - compare nnet and python outputs
- Cleanup excess parameters 